In [1]:
!conda activate DS807

# Exam DS807

## Preparation of data

### Labeling Data and converting images to array values

In [2]:
#import packages for labelling and converting imagery data
import pandas as pd
import numpy as np
import os
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from numpy import asarray

In [3]:
##############
# Func for creating 1D image arrays

def load_image_function(path, question):

    images = [] # empty list placeholder
    labels = [] # empty list placeholder
    container = [] # container to validate correct labels

    for filename in os.listdir(path):
        container.append(filename) # add filename to container
        if question == 'q1':
            CCDY_img = load_img(path + f'/{filename}', target_size = (56, 106),
            color_mode="grayscale")                  
            CCDY_img = img_to_array(CCDY_img)
            images.append(CCDY_img) # append the photo to the images. The images list contains a list of arrays       
        else:
            CCDY_img = load_img(path + f'/{filename}', target_size = (56, 106))        
            CCDY_img = img_to_array(CCDY_img)
            images.append(CCDY_img) # append the photo to the images. The images list contains a list of arrays    

    return asarray(images), container; # not interested in list of arrays, but array containing lists. Asarray does this. Returns three arrays

In [4]:
################
# Func for creating df with classes and 3d img arrays

def load_img_data (question):
    # Start: creatign classes ons string_digits
    # load string digits

    #os.chdir(path_string_digits)

    string_digits = pd.read_csv('DIDA_12000_String_Digit_Labels.csv', 
                 header = None, 
                 names=["index", "string"])
    # create empty class columns
    string_digits['CC'] = 0
    string_digits['D'] = 0
    string_digits['Y'] = 0
    string_digits = string_digits.astype(str)
    # Iterate string digits and append classes
    for i, row in string_digits.iterrows():
        if len(row['string']) != 4:
            row['CC'] = '1'
            row['D'] = '10'
            row['Y'] = '10'
        else:
            row['D'] = row['string'][2]
            row['Y'] = row['string'][3]
            if row['string'][0:2] == '18':
                row['CC']='0'
            else:
                row['CC']='1'
    # End of class labeling on string_digits.
    #
    # Start: create img_df containing scaled images as 1D tensors
    # Convert imagery to 1D arrays with tagged file names

    #os.chdir(path_images)
    image_array, filename = load_image_function('DIDA_12000_String_Digit_Images/DIDA_1', question)

    # and convert to a img_df
    img_df = pd.DataFrame({'filename': filename, 'img_value': list(image_array)}, 
                          columns=['filename', 'img_value'])
    # Create proper index value in img_df to allow merge on string_digits
    img_df['index'] = img_df['filename']
    for i, row in img_df.iterrows():
        row['index'] = str(img_df['index'][i]).split('.')[0]
    img_df
    # End of creating img_df
    #
    # Start: merge img_df with string_digits
    # match index type on dataframes to merge
    string_digits['index'] = string_digits['index'].astype(int)
    img_df['index'] = img_df['index'].astype(int)
    # Merge dataframes
    df_img_classes = string_digits.merge(img_df)
    # Rearrange order of dataframe
    df_img_classes = df_img_classes.reindex(columns= ['index', 'string', 'CC', 'D', 'Y', 'img_value', 'filename'])
    # End og merging data frames
    return df_img_classes


In [5]:
df_q1 = load_img_data('q1')
df_q1['img_value'][0].shape

(56, 106, 1)

In [6]:
df_q2 = load_img_data('q2')
df_q2['img_value'][0].shape

(56, 106, 3)

### Defining fixed train, test and valdiation sets.

In [7]:
df_q1 = df_q1.drop(columns=['index','string','filename'])

df_q2 = df_q2.drop(columns=['index','string','filename'])


### Split train, val, test

#### Subsetting df

In [9]:
gray_train = df_q1.loc[0:7679]
gray_val = df_q1.loc[7680:9599]
gray_test = df_q1.loc[9600:11999]
rgb_train = df_q1.loc[0:7679]
rgb_val = df_q1.loc[7680:9599]
rgb_test = df_q1.loc[9600:11999]


#### Creating labels

In [13]:
#### training labels ###
gray_Y_train = gray_train['Y']
gray_Y_train = gray_Y_train.astype(np.uint8)
gray_D_train = gray_train['D']
gray_D_train = gray_D_train.astype(np.uint8)
gray_CC_train = gray_train['CC']
gray_CC_train = gray_CC_train.astype(np.uint8)

rgb_Y_train = rgb_train['Y']
rgb_Y_train = rgb_Y_train.astype(np.uint8)
rgb_D_train = rgb_train['D']
rgb_D_train = rgb_D_train.astype(np.uint8)
rgb_CC_train = rgb_train['CC']
rgb_CC_train = rgb_CC_train.astype(np.uint8)

#### validation labels ###
gray_Y_val = gray_val['Y']
gray_Y_val = gray_Y_val.astype(np.uint8)
gray_D_val = gray_val['D']
gray_D_val = gray_D_val.astype(np.uint8)
gray_CC_val = gray_val['CC']
gray_CC_val = gray_CC_val.astype(np.uint8)

rgb_Y_val = rgb_val['Y']
rgb_Y_val = rgb_Y_val.astype(np.uint8)
rgb_D_val = rgb_val['D']
rgb_D_val = rgb_D_val.astype(np.uint8)
rgb_CC_val = rgb_val['CC']
rgb_CC_val = rgb_CC_val.astype(np.uint8)

#### test labels ###
gray_Y_test = gray_test['Y']
gray_Y_test = gray_Y_test.astype(np.uint8)
gray_D_test = gray_test['D']
gray_D_test = gray_D_test.astype(np.uint8) ###############
gray_CC_test = gray_test['CC']
gray_CC_test = gray_CC_test.astype(np.uint8)

rgb_Y_test = rgb_test['Y']
rgb_Y_test = rgb_Y_test.astype(np.uint8)
rgb_D_test = rgb_test['D']
rgb_D_test = rgb_D_test.astype(np.uint8)
rgb_CC_test = rgb_test['CC']
rgb_CC_test = rgb_CC_test.astype(np.uint8)

#### Creating image values

In [14]:
#### gray values ####
#gray train
values = np.zeros(shape=(len(gray_train),56,106,1))
i = 0
j = 0
while i < len(gray_train):
    values[i] = gray_train['img_value'][j]
    i += 1
    j+=1
gray_train = values

#gray val
values = np.zeros(shape=(len(gray_val),56,106,1))
i = 0
j = 7680
while i < len(gray_val):
    values[i] = gray_val['img_value'][j]
    i += 1
    j+=1
gray_val = values  

#gray test
values = np.zeros(shape=(len(gray_test),56,106,1))
i = 0
j = 9600
while i < len(gray_test):
    values[i] = gray_test['img_value'][j]
    i += 1
    j+=1
gray_test = values     


#### RGB values ####
#gray train
values = np.zeros(shape=(len(rgb_train),56,106,3))
i = 0
j = 0
while i < len(rgb_train):
    values[i] = rgb_train['img_value'][j]
    i += 1
    j+=1
rgb_train = values

#gray val
values = np.zeros(shape=(len(rgb_val),56,106,3))
i = 0
j = 7680
while i < len(rgb_val):
    values[i] = rgb_val['img_value'][j]
    i += 1
    j+=1
rgb_val = values  

#gray test
values = np.zeros(shape=(len(rgb_test),56,106,3))
i = 0
j = 9600
while i < len(rgb_test):
    values[i] = rgb_test['img_value'][j]
    i += 1
    j+=1
rgb_test = values     



### Data names:



For all values, normaly **x_train**, x_val etc you call:

>gray_val ..etc

>rgb_val ..etc

For all labels, normaly **y_train**, y_val etc you call:

>gray_D_val ..etc

>rgb_CC_val ..etc

## Question 1

### 1.2 Non-deeplearning CC-D-Y models

#### SVM, modelCC

#### Catboost, modelD

#### Catboost, modelY

## Question 2

In [2]:
#Normalization of gray values
gray_val = gray_val/255
gray_train = gray_train/255
gray_test = gray_test/255
rgb_val = rgb_val/255
rgb_train = rgb_train/255
rgb_test = rgb_test/255

NameError: name 'gray_val' is not defined

### 2.2.A Construct a CNN

### 2.2.B Regularization

### 2.2.C Augmentation

### 2.2.D Transfer learning

## 2.3 Prefered CNN model

## Question 3

### 3.1 Visualization of activation maps

### 3.2 Investigating model performance